In [ ]:
import pickle
import os

import pandas as pd
import numpy as np

from pymlpipe.tabular import PyMLPipe
from pymlpipe.pymlpipeUI import start_ui

import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import MinMaxScaler
import optuna

from helper import objective
from helper import get_model 
from helper import evaluate_for_testing

In [ ]:
REPOSITORY_DATA_RAW = 'data raw'
REPOSITORY_DATA_PREPROCESSED = 'data preprocessed'
REPOSITORY_STUDIES = 'studies'

In [ ]:
mlp = PyMLPipe()

# Load data

In [ ]:
class ForexDataset(Dataset):
    
    def __init__(self, X_date, X_now, X_previous_hour, X_previous_day, X_previous_week, X_previous_month, y_bid, y_ask, idx, 
                 n_previous_hour_values, n_previous_day_values, n_previous_week_values, n_previous_month_values, n_features):
        
        self.X_date = X_date[idx].astype(np.int32)
        self.X_now = X_now[idx].astype(np.float32)
        self.X_previous_hour = X_previous_hour[idx].astype(np.float32)
        self.X_previous_day = X_previous_day[idx].astype(np.float32)
        self.X_previous_week = X_previous_week[idx].astype(np.float32)
        self.X_previous_month = X_previous_month[idx].astype(np.float32)
        self.y_bid = y_bid[idx].astype(np.float32)
        self.y_ask = y_ask[idx].astype(np.float32)
        self.n_previous_hour_values = n_previous_hour_values
        self.n_previous_day_values = n_previous_day_values
        self.n_previous_week_values = n_previous_week_values
        self.n_previous_month_values = n_previous_month_values
        self.n_features = n_features
        
    def __len__(self):
        
        return self.y_bid.shape[0]
    
    def __getitem__(self, idx):
        
        return self.X_date[idx], self.X_now[idx], self.X_previous_hour[idx], self.X_previous_day[idx], \
               self.X_previous_week[idx], self.X_previous_month[idx], self.y_bid[idx], self.y_ask[idx]
    
    def fit_scalers(self, scaler_now, scaler_previous_hour, scaler_previous_day, scaler_previous_week, scaler_previous_month, scaler_y_bid, scaler_y_ask):
        
        return scaler_now.fit(self.X_now), scaler_previous_hour.fit(self.X_previous_hour), scaler_previous_day.fit(self.X_previous_day), \
               scaler_previous_week.fit(self.X_previous_week), scaler_previous_month.fit(self.X_previous_month), \
               scaler_y_bid.fit(np.expand_dims(self.y_bid, axis=1)), scaler_y_ask.fit(np.expand_dims(self.y_ask, axis=1))
    
    def scale(self, scaler_now, scaler_previous_hour, scaler_previous_day, scaler_previous_week, scaler_previous_month, scaler_y_bid, scaler_y_ask):
        
        self.X_now = scaler_now.transform(self.X_now)
        self.X_previous_hour = scaler_previous_hour.transform(self.X_previous_hour).reshape(self.X_previous_hour.shape[0], self.n_previous_hour_values, self.n_features)
        self.X_previous_day = scaler_previous_day.transform(self.X_previous_day).reshape(self.X_previous_day.shape[0], self.n_previous_day_values, self.n_features)
        self.X_previous_week = scaler_previous_week.transform(self.X_previous_week).reshape(self.X_previous_week.shape[0], self.n_previous_week_values, self.n_features)
        self.X_previous_month = scaler_previous_month.transform(self.X_previous_month).reshape(self.X_previous_month.shape[0], self.n_previous_month_values, self.n_features)
        self.y_bid = np.squeeze(scaler_y_bid.transform(np.expand_dims(self.y_bid, axis=1)))
        self.y_ask = np.squeeze(scaler_y_ask.transform(np.expand_dims(self.y_ask, axis=1)))
        
    def transfer_to_tensor(self):
        
        self.X_date = torch.from_numpy(self.X_date)
        self.X_now = torch.from_numpy(self.X_now)
        self.X_previous_hour = torch.from_numpy(self.X_previous_hour)
        self.X_previous_day = torch.from_numpy(self.X_previous_day)
        self.X_previous_week = torch.from_numpy(self.X_previous_week)
        self.X_previous_month = torch.from_numpy(self.X_previous_month)
        self.y_bid = torch.from_numpy(self.y_bid)
        self.y_ask = torch.from_numpy(self.y_ask)
        
    def cuda(self):
        
        self.X_date = self.X_date.cuda()
        self.X_now = self.X_now.cuda()
        self.X_previous_hour = self.X_previous_hour.cuda()
        self.X_previous_day = self.X_previous_day.cuda()
        self.X_previous_week = self.X_previous_week.cuda()
        self.X_previous_month = self.X_previous_month.cuda()
        self.y_bid = self.y_bid.cuda()
        self.y_ask = self.y_ask.cuda()
        
    def cpu(self):
        
        self.X_date = self.X_date.cpu()
        self.X_now = self.X_now.cpu()
        self.X_previous_hour = self.X_previous_hour.cpu()
        self.X_previous_day = self.X_previous_day.cpu()
        self.X_previous_week = self.X_previous_week.cpu()
        self.X_previous_month = self.X_previous_month.cpu()
        self.y_bid = self.y_bid.cpu()
        self.y_ask = self.y_ask.cpu()

In [ ]:
dataset_tuning_trains = []
dataset_tuning_validations = []

for i in range(4):
        
    with open(os.path.join(REPOSITORY_DATA_PREPROCESSED, 'dataset_tuning_train_' + str(i) + '.pt'), 'rb') as file:
        dataset_tuning_trains.append(torch.load(file, pickle_module=pickle))
            
    with open(os.path.join(REPOSITORY_DATA_PREPROCESSED, 'dataset_tuning_validation_' + str(i) + '.pt'), 'rb') as file:
        dataset_tuning_validations.append(torch.load(file, pickle_module=pickle))
        
with open(os.path.join(REPOSITORY_DATA_PREPROCESSED, 'dataset_eval_train.pt'), 'rb') as file:
    dataset_eval_train = torch.load(file, pickle_module=pickle)
    
with open(os.path.join(REPOSITORY_DATA_PREPROCESSED, 'dataset_eval_validation.pt'), 'rb') as file:
    dataset_eval_validation = torch.load(file, pickle_module=pickle)
    
with open(os.path.join(REPOSITORY_DATA_PREPROCESSED, 'dataset_eval_test.pt'), 'rb') as file:
    dataset_eval_test = torch.load(file, pickle_module=pickle)

In [ ]:
with open(os.path.join(REPOSITORY_DATA_PREPROCESSED, 'scaler_y_bid.pkl'), 'rb') as file:
    scaler_y_bid = pickle.load(file)
with open(os.path.join(REPOSITORY_DATA_PREPROCESSED, 'scaler_y_ask.pkl'), 'rb') as file:
    scaler_y_ask = pickle.load(file)

# Y bid

In [ ]:
TARGET = 'y_bid'
mlp.set_experiment("Forex EUR CHF Bid")

## MLP 0 layer day memory

In [ ]:
MODEL_NAME = 'MLP0DayMemory'
MODEL_VERSION = 1.0
TUNING_PATIENCE = 5
TUNING_EPOCHS = 100
TESTING_PATIENCE = 10
TESTING_EPOCHS = 50

### Tuning

In [ ]:
if os.path.exists(os.path.join(REPOSITORY_STUDIES, TARGET, 'study ' + TARGET + ' ' + MODEL_NAME + '.pkl')):
    with open(os.path.join(REPOSITORY_STUDIES, TARGET, 'study ' + TARGET + ' ' + MODEL_NAME + '.pkl'), 'rb') as file:
        study = pickle.load(file)
else:
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
    
while True:
    
    study.optimize(
        lambda trial: objective(
            trial, 
            dataset_tuning_trains, 
            dataset_tuning_validations, 
            MODEL_NAME, 
            TARGET, 
            patience=TUNING_PATIENCE, 
            epochs=TUNING_EPOCHS),
        n_trials=1, 
        timeout=None, 
        n_jobs=1)
    
    with open(os.path.join(REPOSITORY_STUDIES, TARGET, 'study ' + TARGET + ' ' + MODEL_NAME + '.pkl'), 'wb') as file:
        pickle.dump(study, file)

### Visualization

In [ ]:
if os.path.exists(os.path.join(REPOSITORY_STUDIES, TARGET, 'study ' + TARGET + ' ' + MODEL_NAME + '.pkl')):
    with open(os.path.join(REPOSITORY_STUDIES, TARGET, 'study ' + TARGET + ' ' + MODEL_NAME + '.pkl'), 'rb') as file:
        study = pickle.load(file)
else:
    raise Exception("Study do not exists")

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_slice(study)

### Evaluation

In [ ]:
if os.path.exists(os.path.join(REPOSITORY_STUDIES, TARGET, 'study ' + TARGET + ' ' + MODEL_NAME + '.pkl')):
    with open(os.path.join(REPOSITORY_STUDIES, TARGET, 'study ' + TARGET + ' ' + MODEL_NAME + '.pkl'), 'rb') as file:
        study = pickle.load(file)
else:
    raise Exception("Study do not exists")

model = get_model(MODEL_NAME, **study.best_params)

model, loss_test_mse, loss_test_mae, loss_test_mse_unscaled, loss_test_mae_unscaled, relative_error, max_error_absolute, max_error_relative, epoch = evaluate_for_testing(
    model, 
    dataset_eval_train, 
    dataset_eval_validation, 
    dataset_eval_test, 
    scaler_target=scaler_y_bid if TARGET == 'y_bid' else scaler_y_ask, 
    target=TARGET, 
    optimizer=study.best_params['optimizer'], 
    batch_size_train=study.best_params['batch_size_train'], 
    batch_size_validation=dataset_eval_validation.__len__(), 
    batch_size_test=dataset_eval_test.__len__(), 
    learning_rate=study.best_params['learning_rate'], 
    weight_decay=study.best_params['weight_decay'], 
    patience=TESTING_PATIENCE, 
    epochs=TESTING_EPOCHS)

mlp.set_version(MODEL_VERSION)

with mlp.run():
    
    mlp.pytorch.register_model(MODEL_NAME, model)
    mlp.log_params({
        "n_previous_hour_values": study.best_params.get("n_previous_hour_values", 0),
        "n_previous_day_values": study.best_params.get("n_previous_day_values", 0),
        "n_previous_week_values": study.best_params.get("n_previous_week_values", 0),
        "n_previous_month_values": study.best_params.get("n_previous_month_values", 0),
        "optimizer": study.best_params['optimizer'],
        "batch_size_train": int(2 ** study.best_params['batch_size_train']),
        "learning_rate": study.best_params['learning_rate'],
        "weight_decay": study.best_params['weight_decay'],
        "patience": TESTING_PATIENCE,
        "epochs": TESTING_EPOCHS,
        "effective epochs": epoch
    })
    mlp.log_metrics({
        "MAE normalized": loss_test_mae,
        "MSE normalized": loss_test_mse,
        "MAE absolute": loss_test_mae_unscaled,
        "MSE absolute": loss_test_mse_unscaled,
        "Relative error": relative_error,
        "Max error absolute": max_error_absolute,
        "Max error relative": max_error_relative
    })

# Y ask

In [ ]:
TARGET = 'y_ask'
mlp.set_experiment("Forex EUR CHF Ask")

# UI MlOps

In [ ]:
start_ui(host='0.0.0.0', port=8085)